In [55]:
list(np.array([1,2,3], dtype='float64'))

[1.0, 2.0, 3.0]

In [118]:
import solana, json
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta as td
from tqdm.notebook import tqdm
import time
from solana.exceptions import SolanaRpcException


In [19]:
from solana.rpc.api import Client

c = Client("https://api.mainnet-beta.solana.com")

address_in_question="MoonRx42874sBfvKw1bgPrCYS8HARfndkUQxX27MyVr"

In [91]:
# print(tx)
def get_tx_balance_change(tx):
    try:
        keys = tx['result']['transaction']['message']['accountKeys']
        balance_deltas =\
            np.array(tx['result']['meta']['postBalances'], dtype='float64') -\
            np.array(tx['result']['meta']['preBalances'], dtype='float64')
        balance_deltas /= 10 ** 9
        if len(keys) == len(balance_deltas):
            balance_deltas = list(balance_deltas)
            return pd.DataFrame(
                {
                    "key": keys,
                    "balance_delta": balance_deltas,
                    "tx_count": [1] * len(keys),
                }
            )
    except KeyError:
        print(tx)

# get_tx_balance_change(tx)

def get_changes_from_transactions(txs):
    return pd.concat([get_tx_balance_change(tx) for tx in txs], ignore_index=True)

# Start executing from here

In [135]:
search_till = dt.now() - td(days=1)

In [136]:
transactions = c.get_signatures_for_address(address_in_question, limit=1000)['result']

transactions_before_filtering = transactions

transactions = [
    tx for tx in transactions
    if dt.fromtimestamp(tx['blockTime']) > search_till
]

if len(transactions) == len(transactions_before_filtering):
    raise Exception('Not enough data for selected time frame, try smaller timeframe')

print(f'found {len(transactions)} transactions')

print('downloading transactions...')
# transactions_details = [c.get_transaction(tx['signature']) for tx in transactions]
transactions_details = []
for tx in tqdm(transactions):
    tx_details = None
    while not tx_details:
        try:
            tx_details = c.get_transaction(tx['signature'])
        except SolanaRpcException as exc_info:
            print(f"Waiting 10s due to too many requests")
            time.sleep(10.0)
    transactions_details.append(tx_details)

balance_changes = get_changes_from_transactions(transactions_details)
print(f"Got {len(balance_changes)} balance changes")

found 264 transactions
downloading transactions...


  0%|          | 0/264 [00:00<?, ?it/s]

Waiting 5s due to too many requests
Waiting 5s due to too many requests
Waiting 5s due to too many requests
Got 810 balance changes


In [139]:
top_changes = balance_changes.groupby("key").sum()

In [140]:
# top growth
top_changes.sort_values(by="balance_delta")[:10]

,balance_delta,tx_count
key,,
Cp5XbkoATWNYFXSPBpzEy9unHywtm4Yb7RF6mExxv8p3,-14.225935,63
TuhvpbzM2dqzw3hkRgM3gAMyLRNGm1qgvU9x6Npj51f,-3.626297,17
5kZXu9i4wkm7VMu5XaLrgDXzkWdziLCn9f3JVp9iBTM8,-3.000030,6
J1H7NUZZu8DNN6WHosbHH4RKWWA39CTaEMg2VfB2boAh,-2.300020,4
2Xn5XGqJHUY4s99prc2zXMm9bowqdBaKJdExguCQ9x3w,-2.222054,3
424DYBD224mT5qHzNyKqKdNDh3RtA74p1S4QEDqhEoN7,-1.034514,8
6Vokuwac5qRdv3W8R4zSX4U4ayxhqL9yjLgj5YAfnN5i,-0.600005,1
5PQs4PS74Nns5RaJwPMExV2ysovWw4YGrUmDyUdHMsBR,-0.565020,4
HGz6USzeTCmsexpFD4aKTgfBYDvt7kgypxxoygRqcbAY,-0.530005,1


In [141]:
# top lose
top_changes.sort_values(by="balance_delta", ascending=False)[:10]

,balance_delta,tx_count
key,,
MoonRx42874sBfvKw1bgPrCYS8HARfndkUQxX27MyVr,27.289466,264
AtjkBwbC6fy9kmKUU4wqFkj4dcudear4fGgVCtomDYoG,1.699625,19
DZDoNuBcrvJ4pddWr4SPwdsuFd3tZLxGZNMDvdvEm1Tc,1.287034,8
EtNeDT7VT2hyU56eKTUX7Ldy7r3kTP9KarEHmybLUwb2,1.225320,3
4yfwH71ec7oHxqJM1v8iXyHZVF1o238xjNLAKaFkofZv,0.883915,7
GpFE35ZkUHKkUjZJDtxwWLcmt1h3QHknDpv6CnVBoJZW,0.881009,2
CzX3j1Kv6vPfdYs2NQ9NT5wtyZp48sU1sKhcbQDy6CMS,0.313760,2
9kviA5p6b7Nehh4Y3gXSDigFd6PSVcutBzdUbdThK9Kv,0.311867,1
D26TRCH35EuDAiksvMVjNDrsDqavL5PQqJvJ9knxkKNp,0.251102,2


In [142]:
# top transactions
top_changes.sort_values(by="tx_count", ascending=False)[:10]

,balance_delta,tx_count
key,,
11111111111111111111111111111111,0.000000,264
MoonRx42874sBfvKw1bgPrCYS8HARfndkUQxX27MyVr,27.289466,264
Cp5XbkoATWNYFXSPBpzEy9unHywtm4Yb7RF6mExxv8p3,-14.225935,63
AtjkBwbC6fy9kmKUU4wqFkj4dcudear4fGgVCtomDYoG,1.699625,19
TuhvpbzM2dqzw3hkRgM3gAMyLRNGm1qgvU9x6Npj51f,-3.626297,17
2WmtNBEsNP7HnMUUoGMcjs11zAmQwVtfjxtCo2zDqY1F,-0.160000,9
424DYBD224mT5qHzNyKqKdNDh3RtA74p1S4QEDqhEoN7,-1.034514,8
Cphc2RijJ3GpUXCML82WSBBGUBeZNAaXt6jCMXYMX5AU,-0.196291,8
DZDoNuBcrvJ4pddWr4SPwdsuFd3tZLxGZNMDvdvEm1Tc,1.287034,8
